In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!cp /gdrive/My\ Drive/task3_dataset.zip /content
!unzip *.zip > /dev/null
!rm *.zip

In [ ]:
import torch
assert torch.__version__.startswith("1.7")
!CC=clang CXX=clang++ python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install split-folders

In [ ]:
!rm -rf task3_dataset/annotation_labeled/
!rm -rf output

In [ ]:
import splitfolders
splitfolders.ratio('task3_dataset', output="leaf", seed=1337, ratio=(.8, 0.2)) 

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

from detectron2.structures import BoxMode

def get_leaf_dicts(img_dir):

    dataset_dicts = []
    for i in os.listdir(os.path.join(img_dir,'annotation')):
        record = {}
        
        filename = os.path.join(img_dir, 'images',i)
        img = cv2.imread(filename)
        height, width = img.shape[:2]

        colors = np.unique(img)[1:]
        
        record["file_name"] = filename
        record["image_id"] = i
        record["height"] = height
        record["width"] = width

        contours=[cv2.findContours(np.where(img == c, 255, 0).astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)[0] for c in colors]
        objs=[]
        for cnts in contours:
            for cnt in cnts:
                cnt=cnt.reshape(-1,2)
                px = cnt[:,0]
                py = cnt[:,1]

                poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                
                obj = {
                    "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [poly],
                    "category_id": 0,
                }
                objs.append(obj)
            del cnt,px,py
        record["annotations"] = objs
        dataset_dicts.append(record)
        del img,contours,cnts
    return dataset_dicts
get_leaf_dicts('task3_dataset/')
# for d in ["train", "val"]:
#     DatasetCatalog.register("leaf_" + d, lambda d=d: get_leaf_dicts("leaf/" + d + "/"))
#     MetadataCatalog.get("leaf_" + d).set(thing_classes=["leaf"])
# leaf_metadata = MetadataCatalog.get("leaf_train")

## Visualize dataset

In [ ]:
dataset_dicts = get_leaf_dicts("leaf/train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=leaf_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from imantics import Mask
import imageio
from matplotlib import pyplot as plt


ara=imageio.imread('task3_dataset/annotation/ara2012_plant001.png')
print(ara.shape)
plt.imshow(ara)